In [58]:
import json
import pandas as pd

file = open(r"D:\BOT_Prime\CIB_DELTA_back\output\data (1).json")
datas = json.load(file)
df = pd.DataFrame(columns=[
        'Name',
        'DOB', 
        'Gender', 
        'FatherName', 
        'CitizenshipDetails', 
        'PassportDetails', 
        'DrivingLicenseDetails',
        'VoterIDDetails',
        'PANDetails', 
        'CompanyDetails',
        'IndianEmbassyDetails',
        'BlackLists',
        'Account_Nature',
        'Status'
    ])
num = 0
for data in datas["BulkOffLoading"]["Individuals"]["Item"]:
    dicts = {}
    data_keys = list(data.keys())
    # print(data_keys)
    for data_key in data_keys[:4]:
        dicts[data_key] = data[data_key]
    for data_key in sorted(data_keys[4:]):
        keys = list(data[data_key].keys())
        data_count = int(data[data_key][keys[0]])
        if data_count > 1:
            details = data[data_key][keys[1]]
            if "BlackList" in data_key:
                dicts[data_key] = ['|'.join(detail.values()) for detail in details]
                continue
            else:  
                for detail in details:
                    dicts[data_key] = '|'.join(detail.values())
                    dicts['Account_Nature'] = 'Individuals'
                    dicts['Status'] = 'new'
                    df.loc[len(df)] = dicts
        elif data_count == 0:
            continue
        else:
            details = data[data_key][keys[1]]
            dicts[data_key] = '|'.join(details.values())

        if "BlackList" not in data_key:
            dicts['Account_Nature'] = 'Individuals'
            dicts['Status'] = 'new'
            df.loc[len(df)] = dicts
            del dicts[data_key]
    num += 1
    if num == 100:
        break

for data in datas["BulkOffLoading"]["Institutions"]["Item"]:
    dicts = {}
    data_keys = list(data.keys())
    for data_key in data_keys:
        if isinstance(data[data_key], str):
            dicts[data_key] = data[data_key]
        else:
            # maintain Citizenship details
            keys = list(data[data_key].keys())
            data_count = int(data[data_key][keys[0]])
            if data_count > 1:
                details = data[data_key][keys[1]]
                dicts[data_key] = ['|'.join(entry.values()) for entry in details]
            elif data_count == 0:
                dicts[data_key] = ""
            else:
                details = data[data_key][keys[1]]
                dicts[data_key] = '|'.join(details.values())

    dicts['Account_Nature'] = 'Institutional'
    dicts['Status'] = 'new'
    num += 1
    if num == 200:
        break

    # appending to df
    df.loc[len(df)] = dicts

In [59]:
df.head()

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,IndianEmbassyDetails,BlackLists,Account_Nature,Status
0,BALRAM PRADHAN,2015-03-15,M,JITENDRA PRADHAN,2827|2046-01-11|Kathmandu,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|5432|2071-01-...,Individuals,new
1,BHADRA BARAILI,2004-11-22,M,SAL BAHADUR BARAILI,2061/30387|2041-08-29|Sankhuwasabha,NaN,NaN,NaN,NaN,NaN,NaN,Bank and Financial Institutions|15073|2076-07-...,Individuals,new
2,SUDHA DEVI DARUKA,2030-01-01,F,SAWAR MAL RANGUTA,10523|2068-08-04|Parsa,NaN,NaN,NaN,NaN,NaN,NaN,Bank and Financial Institutions|14770|2076-06-...,Individuals,new
3,RAJESH RAJ VERMA,2030-08-22,M,KRISHNA RAJ VERMA,5237|2047-07-20|Saptari,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|13189|2076-03...,Individuals,new
4,NEELAM RIJAL,2029-05-21,F,TARAK PRASAD RIJAL,22961|2052-08-13|Morang,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|11812|2075-09...,Individuals,new


In [60]:
df.to_excel('dataaaa.xlsx')

In [61]:
# df.tail(20)

In [62]:
new_columns = []
for i, data in df.iterrows():
    # print(type(data))
    if data['Account_Nature'] == 'Institutional':
        if 'PANDetails' in data:
            if isinstance(data['PANDetails'], list):
                for pan_element in data['PANDetails']:
                    if isinstance(data['CompanyDetails'],list):
                        for company_element in data['CompanyDetails']:
                            new_data = {
                                'Name': data['Name'],
                                'DOB': data['DOB'],
                                'Gender': data['Gender'],
                                'FatherName': data['FatherName'],
                                'CitizenshipDetails': data['CitizenshipDetails'],
                                'PassportDetails': data['PassportDetails'],
                                'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                                'VoterIDDetails': data['VoterIDDetails'],
                                'PANDetails': pan_element,
                                'CompanyDetails': company_element,
                                'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                                'BlackLists': data['BlackLists'],
                                'Account_Nature': data['Account_Nature'],
                                'Status':data['Status']
                            }
                            new_columns.append(new_data)
                    elif isinstance(data['CompanyDetails'],str):
                        new_data = {
                                'Name': data['Name'],
                                'DOB': data['DOB'],
                                'Gender': data['Gender'],
                                'FatherName': data['FatherName'],
                                'CitizenshipDetails': data['CitizenshipDetails'],
                                'PassportDetails': data['PassportDetails'],
                                'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                                'VoterIDDetails': data['VoterIDDetails'],
                                'PANDetails': pan_element,
                                'CompanyDetails': data['CompanyDetails'],
                                'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                                'BlackLists': data['BlackLists'],
                                'Account_Nature': data['Account_Nature'],
                                'Status':data['Status']
                            }
                        new_columns.append(new_data)
            elif isinstance(data['PANDetails'], str):
                if isinstance(data['CompanyDetails'],str):
                    new_data = {
                                    'Name': data['Name'],
                                    'DOB': data['DOB'],
                                    'Gender': data['Gender'],
                                    'FatherName': data['FatherName'],
                                    'CitizenshipDetails': data['CitizenshipDetails'],
                                    'PassportDetails': data['PassportDetails'],
                                    'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                                    'VoterIDDetails': data['VoterIDDetails'],
                                    'PANDetails': data['PANDetails'],
                                    'CompanyDetails': data['CompanyDetails'],
                                    'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                                    'BlackLists': data['BlackLists'],
                                    'Account_Nature': data['Account_Nature'],
                                    'Status':data['Status']
                                }
                    new_columns.append(new_data)
                elif isinstance(data['CompanyDetails'],list):
                    for company_element in data['CompanyDetails']:
                            new_data = {
                                'Name': data['Name'],
                                'DOB': data['DOB'],
                                'Gender': data['Gender'],
                                'FatherName': data['FatherName'],
                                'CitizenshipDetails': data['CitizenshipDetails'],
                                'PassportDetails': data['PassportDetails'],
                                'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                                'VoterIDDetails': data['VoterIDDetails'],
                                'PANDetails': data['PANDetails'],
                                'CompanyDetails': company_element,
                                'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                                'BlackLists': data['BlackLists'],
                                'Account_Nature': data['Account_Nature'],
                                'Status':data['Status']
                            }
                            new_columns.append(new_data)

        else:
            if isinstance(data['CompanyDetails'],str):
                    new_data = {
                                    'Name': data['Name'],
                                    'DOB': data['DOB'],
                                    'Gender': data['Gender'],
                                    'FatherName': data['FatherName'],
                                    'CitizenshipDetails': data['CitizenshipDetails'],
                                    'PassportDetails': data['PassportDetails'],
                                    'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                                    'VoterIDDetails': data['VoterIDDetails'],
                                    'PANDetails': data['PANDetails'],
                                    'CompanyDetails': data['CompanyDetails'],
                                    'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                                    'BlackLists': data['BlackLists'],
                                    'Account_Nature': data['Account_Nature'],
                                    'Status':data['Status']
                                }
                    new_columns.append(new_data)
            elif isinstance(data['CompanyDetails'],list):
                for company_element in data['CompanyDetails']:
                        new_data = {
                            'Name': data['Name'],
                            'DOB': data['DOB'],
                            'Gender': data['Gender'],
                            'FatherName': data['FatherName'],
                            'CitizenshipDetails': data['CitizenshipDetails'],
                            'PassportDetails': data['PassportDetails'],
                            'DrivingLicenseDetails': data['DrivingLicenseDetails'],
                            'VoterIDDetails': data['VoterIDDetails'],
                            'PANDetails': data['PANDetails'],
                            'CompanyDetails': company_element,
                            'IndianEmbassyDetails': data['IndianEmbassyDetails'],
                            'BlackLists': data['BlackLists'],
                            'Account_Nature': data['Account_Nature'],
                            'Status':data['Status']
                        }
                        new_columns.append(new_data)
    elif data['Account_Nature'] == 'Individuals':
        new_data = {
        'Name': data['Name'],
        'DOB': data['DOB'],
        'Gender': data['Gender'],
        'FatherName': data['FatherName'],
        'CitizenshipDetails': data['CitizenshipDetails'],
        'PassportDetails': data['PassportDetails'],
        'DrivingLicenseDetails': data['DrivingLicenseDetails'],
        'VoterIDDetails': data['VoterIDDetails'],
        'PANDetails': data['PANDetails'],
        'CompanyDetails': data['CompanyDetails'],
        'IndianEmbassyDetails': data['IndianEmbassyDetails'],
        'BlackLists': data['BlackLists'],
        'Account_Nature': data['Account_Nature'],
        'Status':data['Status']
    }
    new_columns.append(new_data)
        # print(new_columns)
    # if isinstance(data[])

new_df = pd.DataFrame(new_columns)
# new_df

In [63]:
new_df.head()

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,IndianEmbassyDetails,BlackLists,Account_Nature,Status
0,BALRAM PRADHAN,2015-03-15,M,JITENDRA PRADHAN,2827|2046-01-11|Kathmandu,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|5432|2071-01-...,Individuals,new
1,BHADRA BARAILI,2004-11-22,M,SAL BAHADUR BARAILI,2061/30387|2041-08-29|Sankhuwasabha,NaN,NaN,NaN,NaN,NaN,NaN,Bank and Financial Institutions|15073|2076-07-...,Individuals,new
2,SUDHA DEVI DARUKA,2030-01-01,F,SAWAR MAL RANGUTA,10523|2068-08-04|Parsa,NaN,NaN,NaN,NaN,NaN,NaN,Bank and Financial Institutions|14770|2076-06-...,Individuals,new
3,RAJESH RAJ VERMA,2030-08-22,M,KRISHNA RAJ VERMA,5237|2047-07-20|Saptari,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|13189|2076-03...,Individuals,new
4,NEELAM RIJAL,2029-05-21,F,TARAK PRASAD RIJAL,22961|2052-08-13|Morang,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|11812|2075-09...,Individuals,new


In [64]:
new_df.to_excel('instit.xlsx',index=False)

In [65]:
new_df.head(5)

,Name,DOB,Gender,FatherName,CitizenshipDetails,PassportDetails,DrivingLicenseDetails,VoterIDDetails,PANDetails,CompanyDetails,IndianEmbassyDetails,BlackLists,Account_Nature,Status
0,BALRAM PRADHAN,2015-03-15,M,JITENDRA PRADHAN,2827|2046-01-11|Kathmandu,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|5432|2071-01-...,Individuals,new
1,BHADRA BARAILI,2004-11-22,M,SAL BAHADUR BARAILI,2061/30387|2041-08-29|Sankhuwasabha,NaN,NaN,NaN,NaN,NaN,NaN,Bank and Financial Institutions|15073|2076-07-...,Individuals,new
2,SUDHA DEVI DARUKA,2030-01-01,F,SAWAR MAL RANGUTA,10523|2068-08-04|Parsa,NaN,NaN,NaN,NaN,NaN,NaN,Bank and Financial Institutions|14770|2076-06-...,Individuals,new
3,RAJESH RAJ VERMA,2030-08-22,M,KRISHNA RAJ VERMA,5237|2047-07-20|Saptari,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|13189|2076-03...,Individuals,new
4,NEELAM RIJAL,2029-05-21,F,TARAK PRASAD RIJAL,22961|2052-08-13|Morang,NaN,NaN,NaN,NaN,NaN,NaN,[Bank and Financial Institutions|11812|2075-09...,Individuals,new


In [66]:
new_df_no_duplicates = new_df[~new_df.duplicated(subset=new_df.columns.difference(['BlackLists']), keep='first')]


In [67]:
new_df_no_duplicates.count()

Name                     253
DOB                      135
Gender                   135
FatherName               135
CitizenshipDetails       128
PassportDetails            0
DrivingLicenseDetails      0
VoterIDDetails             1
PANDetails               124
CompanyDetails           118
IndianEmbassyDetails       0
BlackLists               253
Account_Nature           253
Status                   253
dtype: int64

In [68]:
new_df_no_duplicates.to_excel('No_Duplicate.xlsx',index=False)